# Imports

In [ ]:
# Move to the root
import os
cwd = os.getcwd()
if os.path.basename(cwd) != "cv-in-farming":
    os.chdir("../")
print(os.getcwd())

## Libraries

In [ ]:
from collections import OrderedDict
import random

from matplotlib import colors

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Import Utility Functions 

Importing from [helpers.py](../utils/helpers.py).

In [ ]:
from utils.helpers import *

## Import Image Processing Functions

Importing from [image_processing.py](../src/image_processing.py).

In [ ]:
from src.image_processing import *

# Load Frames

In [ ]:
folders = ["test_data/complete/20210309_140832"] # os.listdir(dataset)

# folder_frame_map: Dict of folders -> Dict of frames -> (RGB, D-RGB, ndarray)
folder_frame_map = OrderedDict()

for folder in folders:
    files = os.listdir(folder)
    
    rgb_im_files = []
    depth_arr_files = []
    depth_im_files = []
    timestamps = []
    
    # Filter files wrt their extension
    for file in files:
        if file.endswith("rgb.png"):
            rgb_im_files.append(file)
        elif file.endswith("depth.npy"):
            depth_arr_files.append(file)
        elif file.endswith("depth.png"):
            depth_im_files.append(file)
        elif file.endswith(".json"):
            timestamps.append(file)
        elif file.endswith("edge_vis.png"):
            pass
        elif file.endswith("edge_pts.npy"):
            pass
        else:
            raise AssertionError(f"Unrecognized file: {file}")
            
    rgb_im_files = sorted(rgb_im_files, key=lambda f: int(f.split("_")[0]))
    depth_arr_files = sorted(depth_arr_files, key=lambda f: int(f.split("_")[0]))
    #depth_im_files = sorted(depth_im_files, key=lambda f: int(f.split("_")[0]))
    #folder_frame_map[folder] = list(zip(rgb_im_files, depth_arr_files, depth_im_files))
    folder_frame_map[folder] = list(zip(rgb_im_files, depth_arr_files))
    
print("Number of folders:", len(folder_frame_map))
for i, k in enumerate(folders):
    print("{}) Loaded {} frames from {}".format(i+1, len(folder_frame_map[folder]), k))

# Detect Corners & Edge

In [ ]:
front_config = {
    "out": "test_data/complete/20210309_140832_detections",
    "template_size": 30,
    "position": 1,
    "start_depth": 0.92,  # Given in depth-scale
    "contour_width": 25, # Given in y-scale
    "y_step": 5,         # Given in y-scale
    "n_contours": 1000,
    "ransac_thresh": 30, #15
    "score_thresh": None,
    "roi": [None,None,250,None], # min_y:max_y, min_x:max_x
    "fit_type": "curve",
    "verbose": 0
}

back_config = {
    "out": None,
    "template_size": 30,
    "position": 2,
    "start_depth": 1.10,  # Given in depth-scale
    "contour_width": 25, # Given in y-scale
    "y_step": 5,         # Given in y-scale
    "n_contours": 1000,
    "ransac_thresh": 10,
    "score_thresh": None,
    "roi": [None,None,250,450], # min_y:max_y, min_x:max_x
    "fit_type": "curve",
    "verbose": 0
}

In [ ]:
import gc
def prepare_corner_plot(depth_arr, inliers=None, outliers=None, edge_pixels=None):
    plt.figure(figsize=(10,10))
    depth_arr = np.rint(255 * (depth_arr / depth_arr.max()))
    depth_arr = np.clip(depth_arr * 7, a_min=0, a_max=255).astype(np.uint8)
    plt.imshow(depth_arr, cmap="gray")
    if edge_pixels is not None:
        plt.plot(edge_pixels[:,1], edge_pixels[:,0], color="springgreen", linewidth=2)
    if inliers is not None:
        inlier_pts = plt.scatter(inliers[:,1], inliers[:,0], color="cyan", marker="o")
    if outliers is not None:
        outlier_pts = plt.scatter(outliers[:,1], outliers[:,0], color="red", marker="x")
    if inliers is not None and outliers is not None:
        plt.legend((inlier_pts, outlier_pts), ("inliers", "outliers"), loc=1)
    
def prepare_overlay_plot(image, edge_pixels, cstr="springgreen"):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.plot(edge_pixels[:,1], edge_pixels[:,0], color=cstr, linewidth=2)

def execute(frame_idx, depth_arr, rgb_img, 
            out,
            template_size,
            position,
            start_depth,
            contour_width,
            y_step,
            n_contours,
            ransac_thresh,
            score_thresh,
            roi,
            fit_type,
            verbose):
    
    # Create a template to find corners
    template = create_template(size=template_size, position=position)
    
    # Fit a curve (2nd degree polynomial) to inlier detections
    edge_pixels, inliers, outliers = apply_template_matching(depth_arr,
                                         template,
                                         start_depth=start_depth,     # Given in depth-scale
                                         contour_width=contour_width, # Given in y-scale
                                         y_step=y_step,               # Given in y-scale
                                         n_contours=n_contours,
                                         ransac_thresh=ransac_thresh,
                                         score_thresh=score_thresh,
                                         roi=roi,
                                         fit_type=fit_type,
                                         verbose=verbose)
    
    # Store pixel coordinates for the edge
    if out is not None:
        np.save(f"{out}/{frame_idx}_edge_pts.npy", edge_pixels)
        
    # Visualize or store inlier and outlier corners
#     prepare_corner_plot(depth_arr, inliers, outliers)
#     if out is None:
#         plt.show()
        
    # Visualize or store fitted curve
#     prepare_corner_plot(depth_arr, edge_pixels=edge_pixels)
#     if out is None:
#         plt.show()
      
    # Visualize or store inlier and outlier corners and fitted curve
    prepare_corner_plot(depth_arr, inliers, outliers, edge_pixels)
    if out is None:
        plt.show()
    else:
        plt.savefig(f"{out}/{frame_idx}_edge_vis.png")
        # Clear the current axes.
        plt.cla() 
        # Clear the current figure.
        plt.clf() 
        # Closes all the figure windows.
        plt.close('all')
        gc.collect()

    # Visualize or store the mask overlay on original RGB
#     prepare_overlay_plot(rgb_img, edge_pixels)
#     if out is None:
#         plt.show()
#     else:
#         plt.savefig(f"{out}/rgb_overlay_{frame_idx}.png")
#         # Clear the current axes.
#         plt.cla() 
#         # Clear the current figure.
#         plt.clf() 
#         # Closes all the figure windows.
#         plt.close('all')
#         gc.collect()

## Execute on all frames available

In [ ]:
folder = folders[0]
num_frames = len(folder_frame_map[folder])
x = list(range(num_frames))
# x = x[::400]
for i in x:
    #rgb_im_file, depth_arr_file, depth_im_file = folder_frame_map[folder][i]
    rgb_im_file, depth_arr_file = folder_frame_map[folder][i]
    frame_idx = rgb_im_file.split("_")[0]
    print(f"Detection on frame-{frame_idx}:")
    
    rgb_im_path = os.path.join(folder, rgb_im_file)
    #depth_im_path = os.path.join(folder, depth_im_file)
    depth_arr_path = os.path.join(folder, depth_arr_file)

    rgb_img = cv2.imread(rgb_im_path, cv2.IMREAD_COLOR)     # BGR image
    #depth_img = cv2.imread(depth_im_path, cv2.IMREAD_COLOR) # BGR image
    depth_arr = np.load(depth_arr_path)
    
    execute(frame_idx, depth_arr, rgb_img, **front_config)
    print()

## Pick one frame and execute

In [ ]:
# folder_indices = list(range(8))
# folder_idx = random.choice(folder_indices)
# print("Folder choice:", folder_idx)
folder_idx = 0
folder = folders[folder_idx]

# file_indices = list(range(len(folder_frame_map[folder])))
# i = random.choice(file_indices)
# print("Frame choice:", i)
i = 1287
print("Fetching frame: {} from folder: {}".format(i, folder))

rgb_im_file, depth_arr_file, depth_im_file = folder_frame_map[folder][i]
frame_idx = rgb_im_file.split("_")[0]
print(f"Detection on frame-{frame_idx}:")

rgb_im_path = os.path.join(folder, rgb_im_file)
depth_im_path = os.path.join(folder, depth_im_file)
depth_arr_path = os.path.join(folder, depth_arr_file)

rgb_img = cv2.imread(rgb_im_path, cv2.IMREAD_COLOR)     # BGR image
depth_img = cv2.imread(depth_im_path, cv2.IMREAD_COLOR) # BGR image
depth_arr = np.load(depth_arr_path)

execute(frame_idx, depth_arr, rgb_img, depth_img, **front_config)

In [ ]:
plt.figure(figsize=(10,10))
depth_arr = np.rint(255 * (depth_arr / depth_arr.max()))
adjusted = np.clip(depth_arr * 7, a_min=0, a_max=255).astype(np.uint8)
plt.imshow(adjusted, cmap="gray")

In [ ]:
plt.figure(figsize=(10,10))
adjusted = depth_arr[:,250:]
adjusted = np.rint(255 * (adjusted / depth_arr.max()))
adjusted = np.clip(adjusted * 7, a_min=0, a_max=255).astype(np.uint8)
plt.imshow(adjusted, cmap="gray")

# Generate Video from Frames

In [ ]:
from matplotlib.animation import FFMpegWriter

plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

folder = "front(20201112_140127)"
fig = plt.figure()
cut = folder_frame_map[folder][150:450]

# Fixing random state for reproducibility
metadata = dict(title='Detection Demo')
writer = FFMpegWriter(fps=30, metadata=metadata)

fig = plt.figure()
imgh = plt.imshow(np.zeros((480, 640), dtype=np.uint8))
ph, = plt.plot([], [], color="cyan", linewidth=2)

with writer.saving(fig, "Detection Demo.mp4", 100):
    for rgb_im_file, depth_arr_file, depth_im_file in cut:
        frame_idx = rgb_im_file.split("_")[0]

        rgb_im_path = os.path.join(folder, rgb_im_file)
        rgb_img = cv2.imread(rgb_im_path, cv2.IMREAD_COLOR)
        edge_pixels = np.load(os.path.join(folder, f"{frame_idx}_edge_pts.npy"))
        
        imgh.set_data(rgb_img)
        ph.set_data(edge_pixels[:,1], edge_pixels[:,0])
        
        writer.grab_frame()